In [ ]:
import tdmclient.notebook
from pylsl import StreamInlet, resolve_stream
import numpy as np
import time

In [ ]:
def connectStream():
        # first resolve an EEG stream on the lab network
        print("looking for an EEG stream...")
        streams = resolve_stream('type', 'EEG')


        # create a new inlet to read from the stream
        inlet = StreamInlet(streams[0])

        print('connection found')
        return inlet

In [ ]:
await tdmclient.notebook.start()

In [ ]:
try : inlet = connectStream()
except : print('No stream found')

Code pour suivre la ligne 

In [ ]:
%%run_python

#onevent startup
maxSteer = 20
# speed_left = 400
# speed_right= 400
speed = 0
steerL = 0
tmp = 0

rainbow = [
    32, 0, 0,      # Rouge
    32, 16, 0,     # Orange
    32, 32, 0,     # Jaune
    16, 32, 0,     # Vert clair
    0, 32, 0,      # Vert
    0, 32, 16,     # Turquoise
    0, 32, 32,     # Cyan
    0, 16, 32,     # Bleu clair
    0, 0, 32       # Bleu
]
colorIndex = 0
currentSound = -1

@onevent
def timer0():
    nf_leds_top(rainbow[colorIndex], rainbow[colorIndex + 1], rainbow[colorIndex + 2])
    colorIndex = (colorIndex + 3) % len(rainbow)

def updateColor(s):
    global timer_period, currentSound
    if s == 0:
        #Pas de couleur
        nf_sound_play(-1)
        currentSound = -1
        nf_leds_top(0,0,0)
        timer_period[0] = 0
    elif s <100:
        #Couleur Blue
        nf_sound_play(-1)
        currentSound = -1
        timer_period[0] = 0
        nf_leds_top(0,0,32)
    elif s >= 100 and s < 200:
        # Couleur Vert
        nf_sound_play(-1)
        currentSound = -1
        timer_period[0] = 0
        nf_leds_top(0,32,0)
    elif s >= 200 and s < 300:
        # Couleur Orange
        nf_sound_play(-1)
        currentSound = -1
        timer_period[0] = 0
        nf_leds_top(32,8,0)
    elif s >= 300 and s < 350:
        # Couleur Rouge
        nf_sound_play(-1)
        currentSound = -1
        timer_period[0] = 0
        nf_leds_top(32,0,0)
    elif s >= 350 :
        # Couleur Arc en Ciel
        timer_period[0] = 100
        if currentSound != 7:
            nf_sound_play(7)
            currentSound = 7
        nf_leds_top(32,32,32)

@onevent
def sound_finished():
    global currentSound
    nf_sound_play(currentSound)


@onevent
def sendSpeed(s):
    global speed
    updateColor(s)
    speed = s

@onevent
def prox():
    global speed, motor_left_target, motor_right_target, maxSteer, steerL, tmp
    right = prox_ground_delta[1]
    left = prox_ground_delta[0]
    tmp = maxSteer * left - maxSteer * right

    steerL = tmp // 50

    motor_left_target = speed + steerL
    motor_right_target = speed - steerL


In [ ]:
while True:
        # get a new sample (you can also omit the timestamp part if you're not
    # interested in it)
    sample, timestamp = inlet.pull_sample()
    if timestamp:
        alpha = sample[0]
        # alpha = valeur actuelle - baseline
        # donc si alpha < 0 : plus concentré = on augmente la vitesse
        # sinon on réduit la vitesse
        if alpha > 0:
            speed = max(0,speed - alpha)
        else:
            speed = min(speed - alpha,400)

    # alpha_plot = ax.plot(i, alpha)
    # plt.pause(0.01)  # Mettre à jour le graphique en temps réel

    print(f'Alpha : {alpha}')
    print(f'Speed : {speed}\n')
    send_event('sendSpeed', int(speed))